<a href="https://colab.research.google.com/github/MuhammadIrzam447/NewEncodings/blob/main/Misclassified_Sample_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import os
from PIL import Image
import torch
from torchvision import datasets, transforms
import torchvision.models as models
from torchvision.datasets import ImageFolder
import torchvision
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from PIL import UnidentifiedImageError
from sklearn.metrics import classification_report

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


# Loading Validation Dataset and Preprocessing

In [3]:
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
class ValidationDataset(torch.utils.data.Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.dataset = datasets.ImageFolder(data_dir, transform=val_transform)
        self.classes = sorted(os.listdir(data_dir))
        self.class_lengths = self._compute_class_lengths()
        self.samples = self.dataset.samples  # Store the file paths along with labels


    def _compute_class_lengths(self):
        class_lengths = {cls: 0 for cls in self.classes}

        for cls in self.classes:
            cls_dir = os.path.join(self.data_dir, cls)
            if os.path.isdir(cls_dir):
                class_lengths[cls] = len(os.listdir(cls_dir))

        return class_lengths

    def __getitem__(self, index):
        image, label = self.dataset[index]
        return image, label

    def __len__(self):
        return len(self.dataset)

In [5]:
valPath = "/content/MMLearning/data/food-101/fused_previous/test"
val_dataset = ValidationDataset(valPath)

In [6]:
batch_size = 32
validation_data_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [7]:
print("Number of samples:", len(val_dataset))
print("Number of classes:", len(val_dataset.classes))

Number of samples: 22716
Number of classes: 101


In [8]:
num_batches = len(validation_data_loader)
print("Number of batches:", num_batches)

Number of batches: 710


# Validation

In [ ]:
save_dir = '/content/MMLearning/data/Models/Model-21'
model_name = "fused_model.pth"
load_path = os.path.join(save_dir, model_name)

resnet = torchvision.models.resnet101(pretrained=False)
resnet.fc = nn.Linear(2048, 101) # Choose the number of output classses as per your model
resnet.load_state_dict(torch.load(load_path))

resnet.eval()
resnet.to(device)

In [10]:
# Store misclassified samples
misclassified_samples = []

In [11]:
predicted_classes = []
actual_labels = []

with torch.no_grad():
    for i, (images, labels) in enumerate(validation_data_loader):
        # Move the images and labels to the GPU if available
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = resnet(images)

        # Get the predicted labels
        _, predicted_label = torch.max(outputs, 1)

        # Store the predicted and true labels
        predicted_classes.extend(predicted_label.cpu().tolist())
        actual_labels.extend(labels.cpu().tolist())

        # Check for misclassified samples
        # misclassified_samples.extend([
        #     sample for sample, pred, actual in zip(val_dataset.samples[i * batch_size:(i + 1) * batch_size], predicted_label.cpu(), labels.cpu())
        #     if pred != actual
        # ])
        misclassified_samples.extend([
            (sample[0], pred.item(), actual.item()) for sample, pred, actual in zip(val_dataset.samples[i * batch_size:(i + 1) * batch_size], predicted_label.cpu(), labels.cpu())
            if pred != actual
        ])


In [12]:
len(misclassified_samples)

2018

In [13]:
misclassified_samples[0]

('/content/MMLearning/data/food-101/fused_previous/test/apple_pie/apple_pie_12.jpg.png',
 42,
 0)

In [14]:
%cd /content/MMLearning/data/food-101/fused_previous
with open("fused.txt", "w") as file:
    for item in misclassified_samples:
        file.write(str(item) + "\n")

/content/MMLearning/data/food-101/fused_previous


In [15]:
accuracy = accuracy_score(actual_labels, predicted_classes)
precision = precision_score(actual_labels, predicted_classes, average='weighted')
recall = recall_score(actual_labels, predicted_classes, average='weighted')
f1 = f1_score(actual_labels, predicted_classes, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.9111639373129072
Precision: 0.9111268277909752
Recall: 0.9111639373129072
F1-score: 0.9107555902733985


In [16]:
print(classification_report(actual_labels, predicted_classes))

              precision    recall  f1-score   support

           0       0.92      0.93      0.93       234
           1       0.93      0.96      0.95       221
           2       0.94      0.98      0.96       226
           3       0.84      0.90      0.87       222
           4       0.73      0.65      0.69       225
           5       0.95      0.95      0.95       224
           6       0.96      0.90      0.93       224
           7       0.95      0.92      0.94       225
           8       0.95      0.98      0.96       226
           9       0.91      0.89      0.90       214
          10       0.93      0.96      0.95       231
          11       0.90      0.96      0.93       227
          12       0.94      0.94      0.94       230
          13       0.95      0.94      0.94       220
          14       0.92      0.93      0.93       231
          15       0.95      0.93      0.94       227
          16       0.82      0.77      0.80       224
          17       0.93    